In [ ]:
import pandas as pd
import os
import faiss
import json
import copy
import numpy as np

In [ ]:
folder_path = "data/qm9_data"
series_list = []
for i, file_path in enumerate(os.listdir(folder_path)):
    file = os.path.join(folder_path, file_path, "descriptor.json")
    with open(file, "r") as f:
        data = json.load(f)
    series = pd.Series(data)
    series_list.append(series)
    if i % 1000 == 0: print(i)
df = pd.DataFrame(series_list)

In [ ]:
df.to_parquet("data/descriptors.parquet")

In [ ]:
df = pd.read_parquet("data/descriptors.parquet")

In [ ]:
# SCALE DATA
for col in df.drop("SMILES", axis=1).columns:
    df[col] = (df[col] - df[col].mean()) / df[col].std()

In [ ]:
features = copy.deepcopy(df).drop("SMILES", axis=1)

features["MolWt"] *= 8
features["LogP"] *= 4
features["HOMO"] *= 2
features["LUMO"] *= 2
features["HDonors"] *= 2
features["HAcceptors"] *= 2

In [ ]:
index = faiss.IndexFlatL2(len(features.columns))  
index.add(features)

distances, indices = index.search(features, 1000)

In [ ]:
np.save(os.path.join("data", "indices"), indices)
np.save(os.path.join("data", "distances"), distances)